In [1]:
import joblib

### Load dataset

In [3]:
data_list = joblib.load("../INFO-RAG/uns_data")

In [4]:
len(data_list)

2046737

### only get 100 for testing

In [5]:
data = data_list[:100]
len(data)
#len(data)

100

### InfoRAG functions

In [9]:
import transformers
from torch.utils.data import Dataset
from dataclasses import dataclass

IGNORE_INDEX = -100

def tokenize(
        prompt,
        completion,
        tokenizer: transformers.PreTrainedTokenizer,
):
    """Preprocess the data by tokenizing."""
    source_output = tokenizer.encode(prompt)
    input_seq = prompt + ' ' + completion
    passage_list = prompt
    tokenize_output = tokenizer(input_seq, padding=False, return_tensors=None,max_length=512,truncation=False)
    passage_list_tokenize_output = tokenizer(passage_list, padding=False, return_tensors=None, max_length=512, truncation=False)
    IGNORE_INDEX = -100
    source_len = len(source_output) - 1

    tokenize_output["labels"] = copy.deepcopy(tokenize_output["input_ids"])
    tokenize_output["labels"] = [IGNORE_INDEX] * source_len + tokenize_output["labels"][source_len:]
    return passage_list_tokenize_output,tokenize_output

special_token_list = [1,32000,32001]
import random
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self,
                 tokenizer: transformers.PreTrainedTokenizer,
                 data_type,
                 data_list):
        super(SupervisedDataset, self).__init__()
        self.tokenizer = tokenizer
        self.data_list = data_list

        if data_type == 'train':
            self.data_list = self.data_list[:int(1.0*len(self.data_list))]
        else:
            self.data_list = self.data_list[int(0.2*len(self.data_list))+1:]

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, i):
        if i % (1000) == 0 and int(-1) == 0:
            sp = subprocess.Popen(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            out_str = sp.communicate()
            for out_element in out_str:
                for line in str(out_element).split('\\n'):
                    print(line, file=sys.stderr)
        return self.data_list[i]
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances):
        return instances


In [42]:
# from transformers import AutoTokenizer

# model_name_or_path = "meta-llama/Llama-3.1-8B"

# tokenizer = AutoTokenizer.from_pretrained(
#   model_name_or_path, fast_tokenizer=True)
# tokenizer.pad_token = tokenizer.eos_token
# # make sure tokenizer is right pad in our logic
# tokenizer.padding_side = 'right'


### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [11]:
print(tokenizer.padding_side)
print(tokenizer.pad_token_type_id)
print(tokenizer.eos_token_id)
print(tokenizer.bos_token_id)

right
0
128001
128000


In [12]:
tokenizer.add_tokens(['[MASK_PASSAGE]'])

1

### Dataset

In [13]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import math

train_dataset = SupervisedDataset(tokenizer=tokenizer,data_type='train',data_list=data)
train_sampler = SequentialSampler(train_dataset)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

train_dataloader = DataLoader(train_dataset,
                                  collate_fn=data_collator,
                                  sampler=train_sampler,
                                  shuffle=False,
                                  batch_size=4)

num_update_steps_per_epoch = math.ceil(
        len(train_dataloader) / 0.1) # the higher the lower steps/epoch

num_update_steps_per_epoch

250

### Data Transform

In [14]:
print(tokenizer.decode([26757, 279, 86716, 369, 5678, 323, 813, 8762, 49446, 11, 323, 568, 6244, 279, 1176, 5234, 50003, 315, 279, 6921, 3695, 13, 2028, 2262, 374, 8066, 4184, 311, 856, 6677, 323, 510, 82176, 60, 1396, 220, 23]))


 Aaron the priesthood for himself and his male descendants, and he became the first High Priest of the Israelites.This content is generated according to my knowledge and [REFERENCE] number 8


In [86]:
import os
import copy
from tqdm import tqdm

instances_new = []
data_tag = 0

datasets = []
for step, data_list in tqdm(enumerate(train_dataloader)):
    ### data start ###
    instances_new = []
    data_tag = 0
    for i in range(len(data_list)):
        if step%10 < 4:  # Correct and Complete
            data_tag = 0
            special_token = '[REFERENCE]'
            passage_list = data_list[i][2]
            score_list = data_list[i][3]
            passage_ids_list = []
            for passage in passage_list:
                input_passage_text = (special_token + ' ' + passage)
                output_passage, output_all_token = tokenize(input_passage_text, '',
                                                            tokenizer)
                output_passage_token_ids = output_passage['input_ids']
                label_ids = output_passage_token_ids

                mask_probability = 0.3 #process 30% tokens

                masked_indices = [i for i in range(int(len(output_passage_token_ids) / 3),len(output_passage_token_ids)) if
                                  random.random() < mask_probability]
                masked_ids = []
                idx = 0
                masked_idx = []
                while idx < len(output_passage_token_ids):
                    if (not (idx in masked_indices)) or (output_passage_token_ids[idx] in special_token_list):
                        masked_ids.append(output_passage_token_ids[idx])
                        idx += 1
                    else:  # Process two tokens consecutively
                        rand_num = random.random()
                        if rand_num < 0.5:  # [MASK]
                            masked_ids.append(32000)
                        elif rand_num > 0.5 and rand_num < 0.6:  # Keep
                            masked_ids.append(output_passage_token_ids[idx])
                        else:
                            masked_ids.append(random.randint(3, 31999))  # Replace
                        masked_idx.append(idx)
                        idx += 1
                        if idx < len(output_passage_token_ids) and (
                        not (output_passage_token_ids[idx] in special_token_list)):
                            rand_num = random.random()
                            if rand_num < 0.5:  # [MASK]
                                masked_ids.append(32000)
                            elif rand_num > 0.5 and rand_num < 0.6:  # Keep
                                masked_ids.append(output_passage_token_ids[idx])
                            else:
                                masked_ids.append(random.randint(3, 31999))  # Replace
                            masked_idx.append(idx)
                            idx += 1
                passage_ids_list.append((label_ids, masked_ids))

            query_ids = tokenize(
                'Complete this text according to the above [REFERENCE]: ' + data_list[i][1], '',
                tokenizer)[0]['input_ids'][1:] # remove start token
            selected_s_i = 0
            for s_i in range(len(score_list)):
                if score_list[s_i] == 0:
                    selected_s_i = s_i
                    break
            trace_ids = tokenize(
                'This content is generated according to my knowledge and [REFERENCE] number {}'.format(selected_s_i), '',
                tokenizer)[0]['input_ids'][1:] # remove start token
            start_generation = random.randint(int(len(query_ids) / 2),int((3 * len(query_ids)) / 4))
            answer_label_ids = [IGNORE_INDEX]*start_generation + query_ids[start_generation:] + trace_ids
            input_passage_ids = [1]
            origin_ids = [1]
            for item in passage_ids_list:
                input_passage_ids += item[1][1:]
                origin_ids += item[0][1:]
            input_ids = input_passage_ids + query_ids + trace_ids
            labels = [IGNORE_INDEX] * len(input_passage_ids) + answer_label_ids

            #print("CoC",tokenizer.decode(input_ids),"\n")
            #print("CoC ans",tokenizer.decode(query_ids[start_generation:] + trace_ids),"\n")
            datasets.append(input_ids)

        elif step%10 >= 4 and step%10 < 8:  # Contextual Stimulation
            data_tag = 1
            special_token = '[REFERENCE]'
            passage_list = data_list[i][2]
            selected_passage = '[QUERY] ' + data_list[i][1]
            score_list = data_list[i][3]
            score_list_fuben = []
            for s in score_list:
                if not s == 0:
                    score_list_fuben.append(s)
            if len(score_list) > 1:
                score_list = score_list_fuben
            passage_ids_list = []
            for passage in passage_list:
                input_passage_text = (special_token + ' ' + passage)
                output_passage, output_all_token = tokenize(input_passage_text, selected_passage,
                                                            tokenizer)
                output_passage_token_ids = output_passage['input_ids']
                label_ids = output_passage_token_ids
                if passage == data_list[i][1]:
                    selected_ids = label_ids
                else:
                    passage_ids_list.append(label_ids)

            query_ids = tokenize(
                'Complete this text according to the above [REFERENCE]: ' + data_list[i][1], '',
                tokenizer)[0]['input_ids'][1:] # remove start token
            selected_s_i = 0
            for s_i in range(len(score_list)):
                if score_list[s_i] == 0:
                    selected_s_i = s_i
                    break
            trace_ids = tokenize(
                'This content is generated according to my knowledge'.format(selected_s_i), '',
                tokenizer)[0]['input_ids'][1:] # remove start token
            start_generation = random.randint(int(len(query_ids) / 2),int((3 * len(query_ids)) / 4))
            answer_label_ids = [IGNORE_INDEX]*start_generation + query_ids[start_generation:] + trace_ids
            input_passage_ids = [1]
            origin_ids = [1]
            for item in passage_ids_list:
                input_passage_ids += item[1:]
                origin_ids += item[1:]
            input_ids = input_passage_ids + query_ids + trace_ids
            labels = [IGNORE_INDEX] * len(input_passage_ids) + answer_label_ids

            #print("CS",tokenizer.decode(input_ids),"\n")
            #print("CS ans", tokenizer.decode(query_ids[start_generation:] + trace_ids),"\n")
            datasets.append(input_ids)



        else:  # Select and Copy
            data_tag = 2
            special_token = '[REFERENCE]'
            passage_list = data_list[i][2]
            selected_passage = '[QUERY] ' + data_list[i][1]
            score_list = data_list[i][3]
            passage_ids_list = []
            for passage in passage_list:
                input_passage_text = (special_token + ' ' + passage)
                output_passage, output_all_token = tokenize(input_passage_text, selected_passage,
                                                            tokenizer)
                output_passage_token_ids = output_passage['input_ids']
                label_ids = output_passage_token_ids
                passage_ids_list.append(label_ids)
            query_ids = tokenize(
                'Complete this text according to the above [REFERENCE]: ' + data_list[i][1], '',
                tokenizer)[0]['input_ids'][1:] # remove start token
            selected_s_i = 0
            for s_i in range(len(score_list)):
                if score_list[s_i] == 0:
                    selected_s_i = s_i
                    break
            trace_ids = tokenize(
                'This content is generated according to [REFERENCE] number {}'.format(selected_s_i), '',
                tokenizer)[0]['input_ids'][1:] # remove start token
            start_generation = random.randint(int(len(query_ids) / 2),int((3 * len(query_ids)) / 4))
            answer_label_ids = [IGNORE_INDEX]*start_generation + query_ids[start_generation:] + trace_ids
            input_passage_ids = [1]
            for item in passage_ids_list:
                input_passage_ids += item[1:]
            input_ids = input_passage_ids + query_ids + trace_ids
            labels = [IGNORE_INDEX] * len(input_passage_ids) + answer_label_ids

            #print("SC",tokenizer.decode(input_ids),"\n")
            #print("SC ans", tokenizer.decode(query_ids[start_generation:] + trace_ids),"\n")
            datasets.append(input_ids)
        #print(tokenizer.decode(input_ids))
        #end

25it [00:00, 85.32it/s]


### LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [97]:
from datasets import Dataset
EOS_TOKEN = tokenizer.eos_token
# def formatting_prompts_func(input_text):
#     return { "text" : [text + EOS_TOKEN for text in input_text] }
# #dataset = input_ids.map(formatting_prompts_func, batched = True,)
# dataset = list(map(formatting_prompts_func,input_ids))
# type(dataset)
# Convert to a list of dictionaries
#inputs = tokenizer.decode(datasets)
data = [{"text": tokenizer.decode(t) + EOS_TOKEN} for t in datasets]

# Create a Dataset
dataset = Dataset.from_list(data)
#print(inputs + EOS_TOKEN)
print(dataset['text'][0])
#tokenizer.decode(dataset[99])
#len(dataset)

"[REFERENCE] number 0 Aaron Aaron ( or ; "Ahutility mom")utility Queensutility, high Southernutility and the_Wutility Moses in the Abrahutilityutility.[REFERENCE] number 1 Knowledge of Aaron, along with his brother((*'=>utilityutility from_cardutility, such as the BibleutilityCommand.[REFERENCE] number 2 The Hebrew Bible relates that, unlike Moses, whoutilityutility in}


 Egyptian royal court,utility and his elder sister optional nonatomic remained with their kinsmen in the eastern border-landutility.'
embre jsonObjectosh			 ).[REFERENCE] number 3 When Moses first confronted(savedInstanceState functioning king about the Israelites,entyutility asutilityutility's spokesmanutilitymondphet") toutilityutilityaraoh.[REFERENCE] number 4 Part of the Law (Torah) that Moses.Errorlabels Aaronutility ( or lineceilutilityPlanôn()),
.Linq a prophet, highutility neut and the brother use Mosesutilityquential Ab ignoredutilityutilityGridView[REFERENCE] number 5 Knowledge of Aaron, incrementUST his bro

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        warmup_ratio = 0.1,
        num_train_epochs = 1,

        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)